##  RAG with multi Data Source 

In [7]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [9]:
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
# print(wiki.run("what is LangChain?")) 
print(wiki.name)

wikipedia


In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs =loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
llm = OllamaEmbeddings(model="mistral")

db=FAISS.from_documents(split_docs,llm)
retriever_db =db.as_retriever(search_kwargs={"k": 1})
retriever_db

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206DCAEE300>, search_kwargs={'k': 1})

In [20]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(
    retriever=retriever_db,
    name="LangChainRetriever",
    description="A tool to retrieve documents from LangChain documentation.",
)

In [22]:
retrieval_tool.name



'LangChainRetriever'

In [26]:
#Arxiv tools
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [27]:
tools = [wiki,arxiv, retrieval_tool]


In [28]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\langchain_Models\\End To End Q&A Chatbot_using_Ollam\\chatvenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='LangChainRetriever', description='A tool to retrieve documents from LangChain documentation.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000206DDCDB560>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vecto

In [68]:

from langchain_community.chat_models import ChatOllama  # ✅ Chat model for agents
model = ChatOllama(model="mistral", temperature=0)



In [69]:
## Prompot tempalte
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [ ]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [71]:
agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', "'k'", 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nLangChainRetriever(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langch

In [72]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools=tools, verbose=True)

In [73]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', "'k'", 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nLangChainRetriever(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(ta

In [ ]:
response = agent_executor.invoke({"input": "What is LangChain?"})
print(response)